In [1]:
library(dplyr)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [11]:
df <- data.frame(label = c('a1', 'a2', 'a3', 'a4', 'a5'),
                 x1 = c(1, 2, 3, NA, NA), 
                 x2 = c(100, NA, 300, 400, NA))
df

label,x1,x2
a1,1,100
a2,2,NA
a3,3,300
a4,NA,400
a5,NA,NA


### Get unique values of a column

In [13]:
labels = unique(df$label)
labels
class(labels)

[1] a1 a2 a3 a4 a5
Levels: a1 a2 a3 a4 a5

[1] "factor"

### Access df by column names

In [3]:
df_sub2 <- df[c('x1', 'x2')]
df_sub2

x1,x2
1,100
2,NA
3,300
NA,400
NA,NA


In [4]:
df_sub1 <- df['x1']
df_sub1
class(df_sub1)

x1
1
2
3
NA
NA


[1] "data.frame"

### Set index for df instead of integer index

In [5]:
row.names(df) <- df$label
df

,label,x1,x2
a1,a1,1,100
a2,a2,2,NA
a3,a3,3,300
a4,a4,NA,400
a5,a5,NA,NA


### Remove a column in df

In [6]:
df <- subset(df, select = -c(label))
df

,x1,x2
a1,1,100
a2,2,NA
a3,3,300
a4,NA,400
a5,NA,NA


### remove "rows" in a dataframe with a NA value in one of the columns

In [7]:
df1 = na.omit(df)
df1

,x1,x2
a1,1,100
a3,3,300


### remove "rows" in a dataframe with all NA

In [8]:
df %>% filter_all(any_vars(!is.na(.)))

x1,x2
1,100
2,NA
3,300
NA,400


### Get column names

In [10]:
# col_names = colnames(df)
col_names = names(df)
col_names
class(col_names)

[1] "x1" "x2"

[1] "character"